<a href="https://colab.research.google.com/github/robson-rsp/datascience/blob/main/clustering/book_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Book recommender system
Este projeto é um sistema de recomendação de livros. A técnica utilizada é de filtragem colaborativa baseado em itens. Escolhi essa abordagem pois é mais eficiente que a baseada em usuários. O sistema vai recomendar um número x de livros parecidos com um livro referência, e o único critério de escolha será as notas dadas pelos usuários.

Fonte: https://www.kaggle.com/datasets/rxsraghavagrawal/book-recommender-system

# Initial imports

In [1]:
%pip install ipython-autotime  --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive, files
import pandas as pd
import warnings
drive.mount('/content/drive', force_remount=True)
warnings.filterwarnings("ignore")
%load_ext autotime

Mounted at /content/drive
time: 713 µs (started: 2023-04-10 20:15:25 +00:00)


In [3]:
books   = pd.read_csv("/content/drive/MyDrive/datasets/book-recommender-system/BX-Books.csv", sep=';', encoding='latin-1', on_bad_lines='skip')
ratings = pd.read_csv("/content/drive/MyDrive/datasets/book-recommender-system/BX-Book-Ratings.csv", sep=';', encoding='latin-1', on_bad_lines='skip')

time: 3.01 s (started: 2023-04-10 20:15:25 +00:00)


# EDA

In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


time: 23.1 ms (started: 2023-04-10 20:15:28 +00:00)


In [5]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
time: 590 ms (started: 2023-04-10 20:15:28 +00:00)


Com certeza não vou precisar de todos os atributos. Além disso, a técnica que usarei para criar o sistema dispensa transformações de tipos de dados.

In [6]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


time: 8.62 ms (started: 2023-04-10 20:15:28 +00:00)


In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
time: 231 ms (started: 2023-04-10 20:15:28 +00:00)


# Train/Test split
O correto é fazer a separação do conjunto em treinamento e teste aqui, mas as transformações que farei não correm o risco de causar vazamento de informações(data leak) para o conjunto de teste. Assim, farei a separação após a engenharia de atributos.

# Feature engineering

In [8]:
from scipy.sparse      import csr_matrix
from sklearn.base      import BaseEstimator, TransformerMixin
from sklearn.compose   import ColumnTransformer
from sklearn.pipeline  import Pipeline

import numpy as np

time: 450 ms (started: 2023-04-10 20:15:29 +00:00)


Vou remover alguns atributos que não serão necessário para a tarefa e renomear aqueles que ficarem.  Este projeto tem um processo de engenharia de atributos tão simples que não há necessidade de se criar classes transformadoras ou pipelines, por isso vou usar apenas funções. 

Vou fazer essa separação mesmo que o conjunto não tenha uma label explícita. Na verdade, a label são as notas dos filmes dadas pelos usuários. Uma forma de checar o desempenho de um modelo é avaliar as notas dadas com as notas calculadas. Para isso vou usar o conjunto de teste.

In [9]:
books   = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

books   = books.rename(columns={'ISBN':'isbn', 'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher'})
ratings = ratings.rename(columns={'User-ID':'user_id', 'ISBN':'isbn', 'Book-Rating':'rating'})

print(f'books:   {list(books.columns)}')
print(f'ratings: {list(ratings.columns)}')

books:   ['isbn', 'title', 'author', 'year', 'publisher']
ratings: ['user_id', 'isbn', 'rating']
time: 75.8 ms (started: 2023-04-10 20:15:29 +00:00)


**Etapa ##:** Identifico e seleciono os usuários que tenham feito mais de 100 avaliações e o livros que tenha recebido mais de 50. Os números foram escolhidos seguindo um critério pessoal.


In [10]:
def filter_rows(dataset, col_name, min_ratings):
  valid_rows = dataset[col_name].value_counts() > min_ratings
  valid_rows_ids = valid_rows[valid_rows].index
  mask = dataset[col_name].isin(valid_rows_ids)
  return dataset[mask]


ratings = filter_rows(ratings, 'user_id', 50)
ratings = filter_rows(ratings, 'isbn', 50)

time: 679 ms (started: 2023-04-10 20:15:29 +00:00)


**Etapa ##:** Crio uma nova tabela(livro, usuário) que será preenchida com as notas que cada livro recebeu de cada usuário.

In [11]:
ratings_books = ratings.merge(books, on='isbn').drop(['title', 'author', 'year', 'publisher'], axis=1)
ratings_books = ratings_books.reset_index().drop('index', axis=1)
X = ratings_books.pivot(index='isbn', columns='user_id', values='rating')

time: 266 ms (started: 2023-04-10 20:15:30 +00:00)


**Etapa ##:** Substituo os valores NaN por zero.

In [12]:
X = X.fillna(value=0)

time: 41.3 ms (started: 2023-04-10 20:15:30 +00:00)


**Etapa ##:** Crio uma matriz esparsa que vai ocupar menos espaço do que a original. Isso tornará o treino do modelo mais rápido.

In [13]:
X_sparse = csr_matrix(X)

time: 103 ms (started: 2023-04-10 20:15:30 +00:00)


# Train/Test split

In [18]:
from sklearn.model_selection import train_test_split

time: 576 µs (started: 2023-04-10 20:23:32 +00:00)


# Model training

Como o conjunto de dados não possui rótulo vou usar o modelo de agrupamento(clustering) NearestNeighbors. O seu critério de escolha para decidir quais vetores(linhas) estão mais próximos será a semelhança de cosenos.

In [14]:
from sklearn.neighbors import NearestNeighbors

time: 92.3 ms (started: 2023-04-10 20:15:30 +00:00)


In [15]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(X_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

time: 10.8 ms (started: 2023-04-10 20:15:30 +00:00)


Vou escolher o nome de um livro para que o modelo recomende 10 livros semelhantes em termos de nota.

In [16]:
title = 'Harry Potter'
mask = books['title'].str.contains(title)
book_isbn = books[mask]['isbn']
mask = X.index.isin(book_isbn)
book_reference = X[mask].head(1)

time: 285 ms (started: 2023-04-10 20:15:30 +00:00)


Transformo a linha que contém todas as notas de usuários para o filme selecionado em matriz esparsa. Essa transformação é necessária porque o modelo foi treinado em uma matriz esparsa.

In [17]:
k = 10
distances, indices = model.kneighbors(book_reference.values, n_neighbors=k+1)
for i in indices[0][1:]:
    print(f"Title: {books.iloc[i]['title']}")

Title: Love Ruins Everything: A Novel
Title: Alice's Tulips
Title: Crewel Lye
Title: Last Sword of Power (Stones of Power)
Title: Dragonflight (Dragonriders of Pern Trilogy (Paperback))
Title: Skin: Talking About Sex, Class &amp; Literature
Title: Herr Der Fliegen (Fiction, Poetry and Drama)
Title: Twenty Minute Retreats: Revive Your Spirits in Just Minutes a Day (A Pan Self-discovery Title)
Title: Hen's Teeth
Title: Centaur Aisle
time: 12.8 ms (started: 2023-04-10 20:15:31 +00:00)
